In [2]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Apr_weekend")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [3]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e4
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [4]:
# getting the coefficients of the costs
coeffs_dict_Apr_weekend = readall("../temp_files/coeffs_dict_Apr_weekend.json")
coeffs_dict_Apr_weekend = JSON.parse(coeffs_dict_Apr_weekend)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Apr_weekend["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Apr_weekend["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Apr_weekend["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [5]:
lamb_grid

8-element Array{Float64,1}:
     0.001
     0.01 
     0.1  
     1.0  
    10.0  
   100.0  
  1000.0  
 10000.0  

In [6]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Apr_weekend.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [7]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 720 entries:
  (8,2.5,10.0,3)    => [1077.7818933023943,1176.660070415097,1237.0417606976055…
  (4,1.0,0.01,2)    => [1078.2448580332034,1176.0852605739012,1236.578795966796…
  (4,3.0,0.001,1)   => [2106.8597227140203,759.8073129999999,207.96393128598297…
  (8,2.0,1000.0,3)  => [1075.235587283007,1180.4921360211024,1239.588066716993,…
  (6,1.5,0.1,1)     => [1077.0874462062022,1177.6180868171011,1237.736207793797…
  (6,1.5,100.0,1)   => [1076.3929991099974,1178.9593097798975,1238.430654890002…
  (4,2.0,1000.0,3)  => [1084.9578466297935,1162.6730309478924,1229.865807370206…
  (5,2.5,10.0,1)    => [1084.2633995335987,1164.0142539106992,1230.560254466401…
  (7,0.5,1.0,3)     => [1065.248338216497,1187.8585412070051,1249.8951102368007…
  (5,0.5,100.0,2)   => [1069.6800105133925,1185.3978405821988,1245.143643486607…
  (7,1.5,1.0,3)     => [1074.541140186798,1181.5657749741936,1240.2825138132018…
  (4,0.5,0.01,1)    => [1073.383728359

In [8]:
xl[(8,2.5,0.1,2)]

24-element Array{Float64,1}:
 1087.27 
 1157.38 
 1227.55 
 1518.46 
 1087.27 
 1296.58 
 1218.19 
 1188.96 
 1475.0  
 1373.44 
  881.662
  972.326
 1270.22 
 1285.83 
 1044.73 
  938.299
 1072.76 
 1155.54 
 1325.51 
 1492.51 
 1344.42 
 1131.23 
  983.526
 1221.99 